In [1]:
! pip install geopandas shapely rasterio Pillow imgaug 

In [13]:
import numpy as np
import pandas as pd
import geopandas as gpd
import rasterio
from rasterio.plot import show
from PIL import Image
import random
from pyproj import Proj, transform, Transformer
from sklearn.preprocessing import MinMaxScaler, StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from imblearn.over_sampling import SMOTE
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Dense, Conv2D, Flatten, concatenate, Concatenate
from tensorflow.keras.utils import to_categorical
from keras.optimizers import Adam
import tensorflow as tf
import matplotlib.pyplot as plt
from shapely.geometry import Point
from sklearn.neighbors import NearestNeighbors

# Preparación de datos

In [3]:
ruta_tiff = "imagen.tif"

with rasterio.open(ruta_tiff) as src:
    srs_raster = src.crs.to_proj4()

In [5]:
ruta_csv = "denue.csv"
try:
    df = pd.read_csv(ruta_csv, encoding='utf-8')
except UnicodeDecodeError:
    # If 'utf-8' fails, try 'latin1' encoding
    df = pd.read_csv(ruta_csv, encoding='latin1')

C:\Users\ANDRES.PEREZ\AppData\Local\Temp\ipykernel_2592\95855496.py:6: DtypeWarning: Columns (35) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(ruta_csv, encoding='latin1')


In [7]:
# Selecciona las columnas deseadas
df_limpio = df[["codigo_act", "per_ocu", "latitud", "longitud"]].copy()
df_limpio["codigo_act"] = df_limpio["codigo_act"].astype(str).str.slice(0, 2)
df_limpio["codigo_act"] = df_limpio["codigo_act"].astype(float)
# Reemplaza los valores 32 y 33 por 31
df_limpio['codigo_act'].replace({32: 31, 33: 31}, inplace=True)

# Reemplaza el valor 49 por 48
df_limpio['codigo_act'].replace({49: 48}, inplace=True)

In [8]:
df_limpio = df_limpio.drop('per_ocu', axis=1)

# Convertir a GeoDataFrame

In [10]:
# Crea una columna 'geometry' utilizando las columnas de latitud y longitud
geometry = [Point(xy) for xy in zip(df_limpio['longitud'], df_limpio['latitud'])]
df_limpio['geometry'] = geometry

# Convierte el DataFrame de pandas en un GeoDataFrame de GeoPandas
gdf = gpd.GeoDataFrame(df_limpio, geometry='geometry')


srs_gdf = "+proj=longlat +ellps=WGS84 +datum=WGS84 +no_defs"
transformador = Transformer.from_proj(Proj(srs_gdf), Proj(srs_raster))
gdf["geometry"] = gdf["geometry"].apply(lambda geom: Point(transformador.transform(geom.x, geom.y)))

# Encontrar caracteristicas de los vecinos

In [12]:
import numpy as np
from sklearn.neighbors import NearestNeighbors

# Extrae las coordenadas de 'geometry' y conviértelas en un array numpy de forma vectorizada
coordinates = np.array([[geom.x, geom.y] for geom in gdf['geometry']])

# Crea un objeto NearestNeighbors para buscar los vecinos cercanos
k_neighbors = 5  # Número de vecinos cercanos que deseas considerar
neighbors = NearestNeighbors(n_neighbors=k_neighbors, algorithm='ball_tree', metric='haversine')
neighbors.fit(coordinates)

# Encuentra los vecinos cercanos para todas las filas en un solo paso
distances, indices = neighbors.kneighbors(coordinates)

# Preparar un DataFrame para los códigos de los vecinos
neighbor_codes_df = pd.DataFrame(index=gdf.index, columns=[f'surrounding_code_{i+1}' for i in range(k_neighbors)])

# Asignar los códigos de los vecinos a las nuevas columnas
for i in range(k_neighbors):
    neighbor_codes_df.iloc[:, i] = gdf.iloc[indices[:, i]]['codigo_act'].values

# Concatena las columnas nuevas al GeoDataFrame original
gdf = pd.concat([gdf, neighbor_codes_df], axis=1)


C:\Users\ANDRES.PEREZ\AppData\Local\Temp\ipykernel_2592\2534373260.py:20: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  neighbor_codes_df.iloc[:, i] = gdf.iloc[indices[:, i]]['codigo_act'].values


# Convertir datos a categoricas binarias

In [14]:
# Convert y to a pandas Series
y = gdf['codigo_act'].astype('category')

# Create a dictionary to map category codes to their respective category labels
y_dict = dict(enumerate(y.cat.categories))

# Apply pd.get_dummies to create dummy variables for y
y = pd.get_dummies(y, columns=['codigo_act'])

# Convert y to a numpy array
y = y.values

# Rest of your code remains the same
surronding = ['surrounding_code_1','surrounding_code_2', 'surrounding_code_3', 'surrounding_code_4', 'surrounding_code_5']

# Create an empty DataFrame for X
X = pd.DataFrame()

for i in surronding:
    # Create a new column in X for each column of 'surrounding'
    X[i] = gdf[i].astype('category')

# Apply pd.get_dummies to all columns of X
X = pd.get_dummies(X, columns=surronding)
X = X.values

# Apply SMOTE to the training data for tabular features
smote_tabular = SMOTE(random_state=42, k_neighbors=3)
X_smote, y_smote = smote_tabular.fit_resample(X, y)

In [15]:
X_smote_train, X_smote_test, y_smote_train, y_smote_test = train_test_split(X_smote, y_smote, test_size=0.2, random_state=42)

# Entrenamiento Red

In [16]:
# Crear un modelo de red neuronal para clasificación binaria
model = keras.Sequential([
    keras.layers.Dense(64, activation='relu', input_shape=(X_smote_train.shape[1],)),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(20, activation='sigmoid')  # Una unidad en la capa de salida para clasificación binaria
])

# Compilar el modelo
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])

In [17]:
# Entrenar el modelo
model.fit(X_smote_train, y_smote_train, epochs=100, batch_size=3648, validation_data=(X_smote_test, y_smote_test))

Epoch 1/100
101/101 [==============================] - 1s 9ms/step - loss: 0.1519 - mae: 0.2476 - val_loss: 0.0585 - val_mae: 0.1231
Epoch 2/100
101/101 [==============================] - 1s 6ms/step - loss: 0.0525 - mae: 0.1009 - val_loss: 0.0481 - val_mae: 0.0845
Epoch 3/100
101/101 [==============================] - 1s 7ms/step - loss: 0.0476 - mae: 0.0803 - val_loss: 0.0462 - val_mae: 0.0746
Epoch 4/100
101/101 [==============================] - 1s 7ms/step - loss: 0.0461 - mae: 0.0741 - val_loss: 0.0450 - val_mae: 0.0709
Epoch 5/100
101/101 [==============================] - 1s 6ms/step - loss: 0.0449 - mae: 0.0715 - val_loss: 0.0436 - val_mae: 0.0691
Epoch 6/100
101/101 [==============================] - 1s 6ms/step - loss: 0.0435 - mae: 0.0698 - val_loss: 0.0421 - val_mae: 0.0672
Epoch 7/100
101/101 [==============================] - 1s 6ms/step - loss: 0.0418 - mae: 0.0675 - val_loss: 0.0402 - val_mae: 0.0644
Epoch 8/100
101/101 [==============================] - 1s 6ms/step - 

101/101 [==============================] - 1s 6ms/step - loss: 0.0060 - mae: 0.0097 - val_loss: 0.0058 - val_mae: 0.0083
Epoch 63/100
101/101 [==============================] - 1s 6ms/step - loss: 0.0060 - mae: 0.0096 - val_loss: 0.0058 - val_mae: 0.0082
Epoch 64/100
101/101 [==============================] - 1s 7ms/step - loss: 0.0060 - mae: 0.0095 - val_loss: 0.0058 - val_mae: 0.0082
Epoch 65/100
101/101 [==============================] - 1s 7ms/step - loss: 0.0060 - mae: 0.0094 - val_loss: 0.0058 - val_mae: 0.0081
Epoch 66/100
101/101 [==============================] - 1s 6ms/step - loss: 0.0060 - mae: 0.0094 - val_loss: 0.0057 - val_mae: 0.0081
Epoch 67/100
101/101 [==============================] - 1s 6ms/step - loss: 0.0059 - mae: 0.0092 - val_loss: 0.0057 - val_mae: 0.0080
Epoch 68/100
101/101 [==============================] - 1s 6ms/step - loss: 0.0059 - mae: 0.0092 - val_loss: 0.0057 - val_mae: 0.0079
Epoch 69/100
101/101 [==============================] - 1s 6ms/step - loss:

In [19]:
if tf.__version__ == "2.15.0":
    model.save('output_model/geo_recortes.keras')
else:
    print("Versión de tensorflow no compabile, intenta corriendo:")
    print("podman-compose up model")

Versión de tensorflow no compabile, intenta corriendo:
podman-compose up model


# Aplicacion del modelo

## Convertir coordenadas a un sistema de coordenadas decimal

In [20]:
def latlon_to_gdf(latitud, longitud):
    ruta_tiff = "imagen.tif"

    with rasterio.open(ruta_tiff) as src:
        srs_raster = src.crs.to_proj4()
    # Crea un DataFrame con una sola fila que contiene la latitud y longitud proporcionadas
    data = {'latitud': [latitud], 'longitud': [longitud]}
    df = pd.DataFrame(data)

    # Crea una columna 'geometry' utilizando las columnas de latitud y longitud
    geometry = [Point(xy) for xy in zip(df['longitud'], df['latitud'])]
    df['geometry'] = geometry

    # Crea un GeoDataFrame de GeoPandas
    gdf = gpd.GeoDataFrame(df, geometry='geometry')

        
    srs_gdf = "+proj=longlat +ellps=WGS84 +datum=WGS84 +no_defs"
    transformador = Transformer.from_proj(Proj(srs_gdf), Proj(srs_raster))
    gdf["geometry"] = gdf["geometry"].apply(lambda geom: Point(transformador.transform(geom.x, geom.y)))  
    
    return gdf

# Ejemplo de uso
latitud = 21.908992
longitud = -102.329807
resultado = latlon_to_gdf(latitud, longitud)

## Extraer coordenadas x, y

In [22]:
# Extraer las coordenadas "x" e "y" en columnas separadas
resultado['x'] = resultado['geometry'].x
resultado['y'] = resultado['geometry'].y
[coordenada_x,coordenada_y] = [resultado['x'][0],resultado['y'][0] ]

## Encontrar vecinos cercanos

In [23]:
def find_nearest_neighbors(input_coords, gdf, k_neighbors=5):
    # Extrae las coordenadas del gdf
    coordinates = np.array([[geom.x, geom.y] for geom in gdf['geometry']])

    # Crea e inicializa el objeto NearestNeighbors
    neighbors = NearestNeighbors(n_neighbors=k_neighbors, algorithm='ball_tree', metric='haversine')
    neighbors.fit(coordinates)

    # Encuentra los vecinos más cercanos para las coordenadas dadas
    distances, indices = neighbors.kneighbors([input_coords])

    # Extrae los códigos de actividad de los vecinos más cercanos
    neighbor_codes = gdf.iloc[indices[0]]['codigo_act'].values

    return neighbor_codes

# Uso de la función (ejemplo)
# input_coords = [x, y]  # Reemplaza x, y con las coordenadas de interés
# nearest_neighbor_codes = find_nearest_neighbors(input_coords, gdf)


# Ejemplo de uso de la función:
# Supongamos que tienes un GeoDataFrame llamado 'mi_gdf' y un arreglo de coordenadas llamado 'mis_coordenadas'
neighbors_values = find_nearest_neighbors([coordenada_x,coordenada_y],gdf)

## Codificación de los valores a categoricas

In [24]:
# Diccionario dado
diccionario = {0: 11.0, 1: 21.0, 2: 22.0, 3: 23.0, 4: 31.0, 5: 43.0, 6: 46.0, 7: 48.0, 8: 51.0, 9: 52.0, 10: 53.0, 11: 54.0, 12: 55.0, 13: 56.0, 14: 61.0, 15: 62.0, 16: 71.0, 17: 72.0, 18: 81.0, 19: 93.0}

def crear_vectores(diccionario, neighbors_values):
    # Inicializar la lista de vectores resultantes
    vectores_resultantes = []

    # Iterar sobre cada valor en neighbors_values
    for valor in neighbors_values:
        # Crear un vector de 20 ceros
        vector = [0] * 20

        # Buscar la clave en el diccionario que corresponda al valor actual
        clave = next((k for k, v in diccionario.items() if v == valor), None)

        # Si se encuentra la clave, poner un 1 en la posición correspondiente del vector
        if clave is not None:
            vector[clave] = 1

        # Extender la lista con los elementos del vector
        vectores_resultantes.extend(vector)

    return vectores_resultantes

vectores_resultantes = crear_vectores(diccionario, neighbors_values)
vectores_resultantes = np.array(vectores_resultantes).reshape(-1, 100)

## Predicción

In [27]:
# Realiza las predicciones en tus datos de prueba
predicciones = model.predict(vectores_resultantes)

# Encuentra el índice del valor más alto en cada predicción
indice_predicciones = np.argmax(predicciones, axis=1)

def obtener_numero_por_indice(diccionario, indice):
    # Busca el valor en el diccionario usando el índice dado
    valor = diccionario.get(indice, None)
    return valor


numeros_correspondientes = [obtener_numero_por_indice(diccionario, indice) for indice in indice_predicciones]
diccionario_actividades = {
    11.0: "Agricultura, cría y explotación de animales, aprovechamiento forestal, pesca y caza",
    21.0: "Minería",
    22.0: "Generación, transmisión, distribución y comercialización de energía eléctrica, suministro de agua y de gas natural por ductos al consumidor final",
    23.0: "Construcción",
    31.0: "Industrias manufactureras",
    32.0: "Industrias manufactureras",
    33.0: "Industrias manufactureras",
    43.0: "Comercio al por mayor",
    46.0: "Comercio al por menor",
    48.0: "Transportes, correos y almacenamiento",
    49.0: "Transportes, correos y almacenamiento",
    51.0: "Información en medios masivos",
    52.0: "Servicios financieros y de seguros",
    53.0: "Servicios inmobiliarios y de alquiler de bienes muebles e intangibles",
    54.0: "Servicios profesionales, científicos y técnicos",
    55.0: "Dirección y administración de grupos empresariales o corporativos",
    56.0: "Servicios de apoyo a los negocios y manejo de residuos, y servicios de remediación",
    61.0: "Servicios educativos",
    62.0: "Servicios de salud y de asistencia social",
    71.0: "Servicios de esparcimiento culturales y deportivos, y otros servicios recreativos",
    72.0: "Servicios de alojamiento temporal y de preparación de alimentos y bebidas",
    81.0: "Otros servicios excepto actividades gubernamentales",
    93.0: "Actividades legislativas, gubernamentales, de impartición de justicia y de organismos internacionales y extraterritoriales"
}
actividad = diccionario_actividades.get(numeros_correspondientes[0], "No se encontró la actividad")
print(f"Las coordenadas corresponden muy posiblemente a un negocio de:")
print(actividad)

1/1 [==============================] - 0s 20ms/step
Las coordenadas corresponden muy posiblemente a un negocio de:
Servicios de alojamiento temporal y de preparación de alimentos y bebidas
